In [30]:
%pprint ON #pretty printing
import pdb #debugger

Pretty printing has been turned OFF


#Mining massive datasets week01

##Intro

To solve for simple equations we could use [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination), but for any real life problem we need to use matrix approach, in which flow equation can be written as $r = Mr$. This is similar to eigenvector problem, where r is first egienvector (largest possible value == 1) of stochastic web matrix M (web page x links, j x i). This means we can use **power iteration** to compute r



<a name="Power_Iteration">
##Power iteration
</a>

* initialise with $r_0= [\frac{1}{N} \cdots \frac{1}{N}]$
* iterate $r^{t+1} = Mr^t$ until $| r^{t+1} - r^t | < \varepsilon$ so using L1 norm
    * note that in Matrix M - page on col i point to page on row j.

Lets see this coded:

In [180]:
import math
import numpy as np


#returns converge r, require web matrix M 
def PowerIteration (M):
  #init r as (n,1) matrix
  numPages= M.shape[0]
  r = np.asmatrix(np.ones(numPages)/float(numPages)).T #[1/N...]

  #iterate until converge
  iterationDiff = 1
  while iterationDiff>0:
    r_old = r
    r = M *r
    iterationDiff = sum(r-r_old)[0,0] #convert matrix to float

  return r

In [199]:
#define M for pages (y,a,m)
M = np.matrix([[0.5, 0.5, 0],
               [.5, 0, 1],
               [0, .5, 0]])

NormalRanking = PowerIteration(M)
print(NormalRanking)

[[ 0.41666667]
 [ 0.33333333]
 [ 0.25      ]]


<a name = Random_walk>
##Random walk
</a>
If we consider our algorithm from surfer perspective, we have to include probability in previous equation, so for each epoch t we have $p_{t+1}=Mp_t$. Hence our flow equation can be interpreted as stationary distribution of random walk (that is when $p_{t+1}=p_t$) and from Markov processes we know that, if we fulfil all conditions, stationary distribution is unique.

Given that, does our algorithm always coverage to corect values? Not always:

* spider trap problem - page links to itself absorbing all score importance
* dead end problem - problems with web pages without outgoing links, our score is leaked

###Spider trap problem

In [202]:
#define M with spider trap at page m
M = np.matrix([[0.5, 0.5, 0],
               [.5, 0, 0],
               [0, .5, 1]])

SpiderTrapRanking = PowerIteration(M)
SpiderTrapRanking - NormalRanking 

matrix([[-0.25      ],
        [-0.22916667],
        [ 0.47916667]])

Unlike the example from lecture, floating solutions allow us to avoid (0,0,1) problem but still page m will suck up a 50% rankings from pages a,y. Solution to this problem is to add **teleport** and define probability $1-\beta$ (commoonly 0.1-0.2) when walker will not follow random link but teleport to random page instead.
  
 ###Dead end problem

In [203]:
#define M with dead end at page m
M = np.matrix([[0.5, 0.5, 0],
               [.5, 0, 0],
               [0, .5, 0]])

DeadEndRanking = PowerIteration(M)
DeadEndRanking - NormalRanking 

matrix([[-0.08333333],
        [-0.16666667],
        [-0.08333333]])

This is even worse then before, as our ranking has leaked out and does not longer sum to 1. Solution to this problem is to add **teleport** with probability 1 once we read dead-end. Combining both problems we can rewrite our orginal equation as $r_j = \sum_{i\rightarrow j} \beta \frac{r_i}{d_i}+(1-\beta) \frac{1}{n}$ .

##Explanation of teleports

Folloowing follows markow chains, to do so our matrix M has to be stochastic (all columns sum to 1, teleport allows this for dead ends), irreducible (we cant get stuck in a give state) and aperiodic (there is no repeatability in our data). This is possible by indroducing random jumps, this is  In case of matrix this means introducing new matrix $A = \beta M +(1-\beta) \frac{1}{n}ee^T$, which we then use for flow equation, hence final equation is $r_{t+1}= A r_t$.


##What if it does not fit into memory of our machine?



#Week1 tests
##Question 1
Suppose we compute PageRank with a β of 0.7, and we introduce the additional constraint that the sum of the PageRanks of the three pages must be 3, to handle the problem that otherwise any multiple of a solution will also be a solution. Compute the PageRanks a, b, and c of the three pages A, B, and C, respectively. Then, identify from the list below, the true statement.
![Q1](https://d396qusza40orc.cloudfront.net/mmds/images/otc_pagerank2.gif).

##Solution
To start with lets define final **PowerIteration** function solving for both dead ends and spider traps.

In [32]:
import math
import numpy as np


#returns converge r, require web matrix M, StayProbability, MaxAllowedIterations
#StayProbability, \beta - how often will random walker just follow the link
#otherwwords JumpProbability = 1-StayProbability
def PowerIteration (M,StayProbability,AllowedIterations=300):
  CovergeAccuracy = 10**-5
  #init r as (n,1) matrix
  numPages= M.shape[0]
  r = np.asmatrix(np.ones(numPages)/float(numPages)).T #[1/N...]

  #calculate matrix A
  A = StayProbability*M + np.ones((numPages,numPages))*(1-StayProbability)/float(numPages)
  #pdb.set_trace()

  #iterate until converge
  iterationDiff = 1
  while iterationDiff>CovergeAccuracy:
    r_old = r
    r = A *r
    iterationDiff = sum(r-r_old)[0,0] #convert matrix to float

  return (A,r*numPages) 
#we introduce constrain that PageRank sums to no of pages, as otherwise any multiple of solution is also solution

Then we can define matrix M, remembering that in Matrix M - page on col i point to page on row j. Hence

In [43]:
M = np.matrix([
        [0,0,0],
        [.5,0,0],
        [0.5,1,1]      
    ])
A,PageRank = PowerIteration(M,0.7)
print PageRank
#multiply page rank by number of pages, as sum == 

print "a+c = %.3f" %(PageRank[0]+PageRank[2])
print "b+c = %.3f" %(PageRank[1]+PageRank[2])
print "a+b = %.3f" %(PageRank[0]+PageRank[1])

[[ 0.3 ]
 [ 0.65]
 [ 2.05]]
a+c = 2.350
b+c = 2.700
a+b = 0.950


##Question 2
Consider three Web pages with the following links:
![Q1](https://d396qusza40orc.cloudfront.net/mmds/images/otc_pagerank3.gif).

Suppose we compute PageRank with β=0.85. Write the equations for the PageRanks a, b, and c of the three pages A, B, and C, respectively. Then, identify in the list below, one of the equations.

In [42]:
M = np.matrix([
        [0,0,1],
        [.5,0,0],
        [0.5,1,0]      
    ])
A,PageRank = PowerIteration(M,0.85)
print A

print "a= %.3fb+ %.3fc" %(A[0,1]/(1-A[0,0]),A[0,2]/(1-A[0,0]))
print "b= %.3fa+ %.3fc" %(A[1,0]/(1-A[1,1]),A[1,2]/(1-A[1,1]))
print "c= %.3fa+ %.3fb" %(A[2,0]/(1-A[2,2]),A[2,1]/(1-A[2,2]))

[[ 0.05   0.05   0.9  ]
 [ 0.475  0.05   0.05 ]
 [ 0.475  0.9    0.05 ]]
a= 0.053b+ 0.947c
b= 0.500a+ 0.053c
c= 0.500a+ 0.947b


##Question 3
Consider three Web pages with the following links:
![Q1](https://d396qusza40orc.cloudfront.net/mmds/images/otc_pagerank3.gif).

Assuming no "taxation," compute the PageRanks a, b, and c of the three pages A, B, and C, using iteration, starting with the "0th" iteration where all three pages have rank a = b = c = 1. Compute as far as the 5th iteration, and also determine what the PageRanks are in the limit. Then, identify the true statement from the list below.

#Links to explore